In [1]:
using JuMP
using GLPK
using LinearAlgebra

### Reference
- Integer Programming and Network Flows T.C. Hu
- http://eaton.math.rpi.edu/faculty/Mitchell/courses/matp6620/notesMATP6620/lecture23/23B_bendersdecomposition.pdf
- http://eaton.math.rpi.edu/faculty/Mitchell/courses/matp6620/notesMATP6620/lecture23/23C_benders_egbeamer.pdf

\begin{array}{ll}
\tag{1}
\\maximize & c_1x + c_2y
\\st \\ & A_1x + A_2y \leq b \\
& x \geq 0, y \in Y
\end{array}

For a given value of $y$, problem (1) becomes:

\begin{array}{ll}
\tag{2}
\\maximize & c_1x
\\st \\ & A_1x  \leq b -  A_2y\\
& x \geq 0
\end{array}

The dual problem of (3) is

\begin{array}{ll}
\tag{2}
\\minimize & u(b -  A_2y)
\\st \\ & A_1^Tu  \geq c_1\\
& u \geq 0
\end{array}

### Manual Solution

In [2]:
A1 = [
    1 0 1 0;
    1 0 0 1;
    0 1 1 0;
    0 1 0 1;
    0 0 0 0;
    1 0 0 0;
    0 1 0 0;
    0 0 1 0;
    0 0 0 1
    ]

A2 = [
    0 0;
    0 0;
    0 0;
    0 0;
    -1 -1;
    -1 0;
    -1 0;
    0 -1;
    0 -1  
    ]

b = [1 1 1 1 -1 0 0 0 0]'

c1 = [8 9 5 6]'

c2 = [-15 -10]'

2×1 Adjoint{Int64,Array{Int64,2}}:
 -15
 -10

In [3]:
# DUAL SUBPROBLEM

function DSP(A1, A2, b, c1, y)
    m, n =  size(A2)
    
    dsp = Model(GLPK.Optimizer)
    @variable(dsp, u[1:m] >= 0)
    refCon = @constraint(dsp, A1' * u .>= c1)
    @objective(dsp, Min, LinearAlgebra.dot(u, (b .- A2 * y)))
    
    optimize!(dsp)
    st = termination_status(dsp)
    uVal = value.(u)
    objDSP = JuMP.getobjectivevalue(dsp)
    
    return st, u, uVal, objDSP, refCon
end

DSP (generic function with 1 method)

In [4]:
m, n = size(A2)

master = Model(GLPK.Optimizer)
@variable(master, 0 <= y[1:n] <= 1e6, Bin) # add some limits (problem dependant)
@variable(master, t <= 1e6) # t is unrestricted - add some limits (problem dependant)
@objective(master, Max, LinearAlgebra.dot(c2, y) + t)
master

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: t, y

In [5]:
optimize!(master)

In [6]:
termination_status(master)

OPTIMAL::TerminationStatusCode = 1

In [7]:
value.(y)

2-element Array{Float64,1}:
 0.0
 0.0

In [8]:
JuMP.getobjectivevalue(master)

1.0e6

In [9]:
st, u, uVal, objDSP, refCon = DSP(A1, A2, b, c1, value.(y))

(MathOptInterface.DUAL_INFEASIBLE, VariableRef[u[1], u[2], u[3], u[4], u[5], u[6], u[7], u[8], u[9]], [NaN, NaN, NaN, NaN, 1.0, NaN, NaN, NaN, NaN], 17.0, ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape}[u[1] + u[2] + u[6] >= 8.0; u[3] + u[4] + u[7] >= 9.0; u[1] + u[3] + u[8] >= 5.0; u[2] + u[4] + u[9] >= 6.0])

In [10]:
# Because the problem is DUAL_INFEASIBLE, add feasibility cut to the master
replace!(uVal, NaN => 0.0)

@constraint(master, LinearAlgebra.dot(uVal, (b .- A2 * y)) >= 0)

y[1] + y[2] >= 1.0

In [11]:
optimize!(master)
termination_status(master)

OPTIMAL::TerminationStatusCode = 1

In [12]:
value.(y)

2-element Array{Float64,1}:
 0.0
 1.0

In [13]:
JuMP.getobjectivevalue(master)

999990.0

In [14]:
st, u, uVal, objDSP, refCon = DSP(A1, A2, b, c1, value.(y))

(MathOptInterface.OPTIMAL, VariableRef[u[1], u[2], u[3], u[4], u[5], u[6], u[7], u[8], u[9]], [5.0, 0.0, 0.0, 6.0, 0.0, 3.0, 3.0, 0.0, 0.0], 11.0, ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape}[u[1] + u[2] + u[6] >= 8.0; u[3] + u[4] + u[7] >= 9.0; u[1] + u[3] + u[8] >= 5.0; u[2] + u[4] + u[9] >= 6.0])

In [15]:
JuMP.getdual.(refCon)

4×1 Array{Float64,2}:
 0.0
 0.0
 1.0
 1.0

In [16]:
# objecive value DSP 11 < t = 999990.0 - continue
# problem is optimal, add optimality constraint
replace!(uVal, NaN => 0.0)

@constraint(master, LinearAlgebra.dot(uVal, (b .- A2 * y)) >= t)
master


A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: t, y

In [17]:
optimize!(master)
termination_status(master)

OPTIMAL::TerminationStatusCode = 1

In [18]:
value.(y), value.(t)

([1.0, 0.0], 17.0)

In [19]:
st, u, uVal, objDSP, refCon = DSP(A1, A2, b, c1, value.(y))

(MathOptInterface.OPTIMAL, VariableRef[u[1], u[2], u[3], u[4], u[5], u[6], u[7], u[8], u[9]], [5.0, 3.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0], 17.0, ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape}[u[1] + u[2] + u[6] >= 8.0; u[3] + u[4] + u[7] >= 9.0; u[1] + u[3] + u[8] >= 5.0; u[2] + u[4] + u[9] >= 6.0])

In [20]:
# objecive value DSP 17 == t = 17 - stop - optimal value

### All together

In [21]:
function benders(;A1, A2, b, c1, c2, eps = 0.001)

    m, n = size(A2)

    master = Model(GLPK.Optimizer)
    @variable(master, 0 <= y[1:n] <= 1e6, Bin) # add some limits (problem dependant)
    @variable(master, t <= 1e6) # t is unrestricted - add some limits (problem dependant)
    @objective(master, Max, LinearAlgebra.dot(c2, y) + t)

    optimize!(master)
    t_value = JuMP.value.(t)

    st, u, uVal, objDSP, refCon = DSP(A1, A2, b, c1, value.(y))


    while abs(t_value - objDSP) >= eps

        if st == MOI.DUAL_INFEASIBLE
            println("It is unbounded. Adding feasibility constraint...")
            println("-------------------------------------------------")
            replace!(uVal, NaN => 0.0)
            @constraint(master, LinearAlgebra.dot(uVal, (b .- A2 * y)) >= 0)

        else
            println("It is optimal. Adding optimality constraint...")
            println("-------------------------------------------------")
            replace!(uVal, NaN => 0.0)
            @constraint(master, LinearAlgebra.dot(uVal, (b .- A2 * y)) >= t)
        end

        optimize!(master)
        t_value = JuMP.value.(t)
        st, u, uVal, objDSP, refCon = DSP(A1, A2, b, c1, value.(y))

    end
    
    # solve the primal subproblem to compare
    SP = Model(GLPK.Optimizer)
    @variable(SP, x[1:size(A1)[2]] >= 0)
    @constraint(SP, A1 * x .<= b .- A2 * value.(y))
    @objective(SP, Max, LinearAlgebra.dot(c1, x))
    optimize!(SP)
    
    println("y optimal values \t\t $(value.(y))")
    println("x optimal values \t\t $(JuMP.value.(x))")
    println("x optimal values using dual \t $(JuMP.dual.(refCon))")
    println("optimal value: \t \t \t $(getobjectivevalue(master))")
    
end

benders (generic function with 1 method)

In [22]:
benders(A1 = A1, A2 = A2, b = b, c1 = c1, c2 = c2, eps = 0.001)

It is unbounded. Adding feasibility constraint...
-------------------------------------------------
It is optimal. Adding optimality constraint...
-------------------------------------------------
y optimal values 		 [1.0, 0.0]
x optimal values 		 [1.0, 1.0, 0.0, 0.0]
x optimal values using dual 	 [1.0; 1.0; 0.0; 0.0]
optimal value: 	 	 	 2.0


### Solving original problem

In [23]:
num_x = length(c1) 
num_y = length(c2)

original = Model(GLPK.Optimizer)
@variable(original, x[1:num_x] >= 0)
@variable(original, y[1:num_y] >= 0, Bin)
@constraint(original, A1 * x + A2 * y .<= b)
@objective(original, Max, LinearAlgebra.dot(c1, x) + LinearAlgebra.dot(c2, y))
optimize!(original)

In [24]:
value.(x)

4-element Array{Float64,1}:
 1.0
 1.0
 0.0
 0.0

In [25]:
value.(y)

2-element Array{Float64,1}:
 1.0
 0.0

In [26]:
getobjectivevalue(original)

2.0

### Example from T.C. Hu pag. 263
Transform problem as described above:
- Problem in max form
- Constraints less than equal


In [27]:
A1 = -[1 4 2]'

A2 = -[
    3 2;
    -1 1;
    1 -1
    ]

b = -[5 7 4]'

c1 = -[5]'

c2 = -[2 2]'


2×1 Adjoint{Int64,Array{Int64,2}}:
 -2
 -2

In [28]:
benders(A1 = A1, A2 = A2, b = b, c1 = c1, c2 = c2, eps = 0.001)

It is optimal. Adding optimality constraint...
-------------------------------------------------
It is optimal. Adding optimality constraint...
-------------------------------------------------
y optimal values 		 [1.0, 0.0]
x optimal values 		 [2.0]
x optimal values using dual 	 [2.0]
optimal value: 	 	 	 -12.0
